INSTALL DEPENDENCIES AND READ IN DATAFRAME

In [1]:
!pip install pandas
!pip install dash

In [2]:
import pandas as pd

# URL of the file
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

# Read the file from the URL into a DataFrame
spacex_df = pd.read_csv(url)

# Display the first few rows of the DataFrame
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


TASK 1: Add a Launch Site Drop-down Input 

In [3]:
import dash
from dash import dcc, html, Input, Output
import plotly.express as px

# Assume you already have spacex_df loaded and defined

# Create the Dash application
app2 = dash.Dash(__name__)

# Define launch_site variable
launch_sites = spacex_df['Launch Site'].unique().tolist()
launch_sites.append('All')  # Add 'All' option

# Create the dropdown menu
dropdown_menu = dcc.Dropdown(
    id='launch-site-dropdown',
    options=[{'label': site, 'value': site} for site in launch_sites],
    value='All',  # Default value
)

# Define callback function to update pie chart
@app2.callback(
    Output('success-pie-chart', 'figure'),
    [Input('launch-site-dropdown', 'value')]
)
def update_pie_chart(selected_site):
    if selected_site == 'All':
        # Filter dataframe for successful launches only
        filtered_df = spacex_df[spacex_df['class'] == 1]
        # Calculate total success launches by site
        total_success_launches = filtered_df.groupby('Launch Site').size()
        # Create pie chart
        fig = px.pie(
            names=total_success_launches.index,
            values=total_success_launches.values,
            title='Total Success Launches by Site'
        )
    else:
        # Filter dataframe for selected launch site
        filtered_df = spacex_df[spacex_df['Launch Site'] == selected_site]
        # Calculate success and failure counts
        success_count = (filtered_df['class'] == 1).sum()
        failure_count = (filtered_df['class'] == 0).sum()
        # Create pie chart
        fig = px.pie(
            names=['Success', 'Failure'],
            values=[success_count, failure_count],
            title=f'Success vs Failure for {selected_site}'
        )
    
    fig.update_traces(marker=dict(colors=['green', 'red']))
    
    return fig

# Define calculate_success_rate function
def calculate_success_rate(min_payload, max_payload):
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= min_payload) & 
                            (spacex_df['Payload Mass (kg)'] <= max_payload)]
    total_rows = len(filtered_df)
    success_rows = len(filtered_df[filtered_df['class'] == 1])
    if total_rows == 0:
        return 0
    else:
        return success_rows / total_rows

# Create the layout
app2.layout = html.Div(children=[
    html.H1(children='SpaceX Launch Dashboard'),
    html.Div(children=[
        html.Div(children=[
            html.Label('Launch Site:'),
            dropdown_menu,
        ], className='dropdown-container'),
        # Placeholder for the pie chart
        dcc.Graph(id='success-pie-chart')
    ]),
    # New slider and output section
    html.Div([
        dcc.RangeSlider(
            id='payload-slider',
            min=0,
            max=10000,
            step=500,
            marks={i: str(i) for i in range(0, 10001, 500)},
            value=[0, 10000]
        ),
        html.Div(id='output-success-rate')
    ]),
    # Scatterplot section
    html.Div([
        dcc.Graph(id='scatter-plot')
    ])
])

# Define callback to update success rate output
@app2.callback(
    Output('output-success-rate', 'children'),
    [Input('payload-slider', 'value')]
)
def update_success_rate(payload_range):
    min_payload, max_payload = payload_range
    success_rate = calculate_success_rate(min_payload, max_payload)
    return f"Success Rate: {success_rate:.2%}"

# Define callback to update scatter plot
@app2.callback(
    Output('scatter-plot', 'figure'),
    [Input('launch-site-dropdown', 'value'),
     Input('payload-slider', 'value')]
)
def update_scatter_plot(selected_site, payload_range):
    min_payload, max_payload = payload_range
    if selected_site == 'All':
        filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= min_payload) & 
                                (spacex_df['Payload Mass (kg)'] <= max_payload)]
    else:
        filtered_df = spacex_df[(spacex_df['Launch Site'] == selected_site) & 
                                (spacex_df['Payload Mass (kg)'] >= min_payload) & 
                                (spacex_df['Payload Mass (kg)'] <= max_payload)]
    fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version')
    return fig

# Run the app
if __name__ == '__main__':
    app2.run_server(debug=True, port=8051)
